In [2]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

  Using cached chromadb-1.4.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.2 kB)
  Using cached faiss_cpu-1.13.2-cp310-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (7.6 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Wikipedia Retie rver

In [5]:
from langchain_community.retrievers import WikipediaRetriever

In [9]:
retriever = WikipediaRetriever(top_k_results=2, lang='en')

In [15]:
query = "what is behavioural economics and how can we leverage that to improve decisions"
docs = retriever.invoke(query)

In [16]:
for doc in docs:
    print(doc.page_content)

Behavioral economics is the study of the psychological (e.g. cognitive, behavioral, affective, social) factors involved in the decisions of individuals or institutions, and how these decisions deviate from those implied by traditional economic theory.
Behavioral economics is primarily concerned with the bounds of rationality of economic agents. Behavioral models typically integrate insights from psychology, neuroscience and microeconomic theory.
Behavioral economics began as a distinct field of study in the 1970s and 1980s, but can be traced back to 18th-century economists, such as Adam Smith, who deliberated how the economic behavior of individuals could be influenced by their desires.

The status of behavioral economics as a subfield of economics is a fairly recent development; the breakthroughs that laid the foundation for it were published through the last three decades of the 20th century. Behavioral economics is still growing as a field, being used increasingly in research and in

## Vector Store Retriever

In [17]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [27]:
# Step 1: Your source documents
documents = [
    Document(page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose"),
    Document(page_content="A bunch of scientists bring back dinosaurs and mayhem breaks"),
    Document(page_content="A psychologist / detective gets lost in a series of dreams")
]

In [28]:
embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vector_store = Chroma.from_documents(
    embedding=embedding_model,
    documents=documents,
    collection_name="my_collection_name",
    persist_directory="./chroma_db"
)

In [22]:
retriever = vector_store.as_retriever(search_kwargs={'k':2})

In [24]:
retriever.invoke(query)

[Document(metadata={}, page_content='A psychologist / detective gets lost in a series of dreams'),
 Document(metadata={}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose')]

## Maximum Mariginal Relevance


In [29]:
query = 'dinasaur'
retriever2 = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 3, 'lambda_mult':0.5}
    )

In [30]:
retriever2.invoke(query)

[Document(metadata={}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
 Document(metadata={}, page_content='Leo DiCaprio gets lost in a dream'),
 Document(metadata={}, page_content='A psychologist / detective gets lost in a series of dreams')]